In [1]:

import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Display full outputs
pd.set_option("display.max_columns", None)


In [5]:
# Load the dataset 
df = pd.read_csv("../Data/processed_data_with_target.csv")  

# Check the data
df.head()


,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,total_amount,avg_amount,transaction_count,std_amount,total_value,avg_value,std_value,transaction_hour,transaction_day,transaction_month,transaction_year,TransactionId_woe,BatchId_woe,AccountId_woe,SubscriptionId_woe,CustomerId_woe,CurrencyCode_woe,ProviderId_woe,ProductId_woe,ProductCategory_woe,ChannelId_woe,is_high_risk
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,0.0,ProviderId_6,ProductId_10,airtime,ChannelId_3,-0.046371,-0.072291,2018-11-15 02:18:49+00:00,-0.349252,-0.044962,0.170118,-0.067623,-0.311831,-0.167524,-0.244587,-0.095799,-0.165668,2,15,11,2018,0.099615,0.099615,2.068537,2.068537,2.600077,0.0,3.130871,1.642476,1.619520,-0.472311,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,0.0,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-0.054643,-0.080251,2018-11-15 02:19:08+00:00,-0.349252,-0.044962,0.170118,-0.067623,-0.311831,-0.167524,-0.244587,-0.095799,-0.165668,2,15,11,2018,0.099615,0.099615,4.115579,4.170266,2.600077,0.0,2.733265,4.170420,-0.565355,2.705442,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,0.0,ProviderId_6,ProductId_1,airtime,ChannelId_3,-0.050426,-0.076352,2018-11-15 02:44:21+00:00,-0.349252,-0.044962,0.165122,-0.072568,-0.444993,-0.201719,-0.254056,-0.109992,-0.195719,2,15,11,2018,0.099615,0.099615,0.190201,0.190201,0.190201,0.0,3.130871,5.714367,1.619520,-0.472311,1
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,0.0,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,0.107717,0.096648,2018-11-15 03:32:55+00:00,-0.349252,-0.044962,0.175567,-0.008155,-0.404020,-0.008737,-0.240722,-0.027588,0.003014,3,15,11,2018,0.099615,0.099615,1.314636,1.314636,1.605500,0.0,-1.617530,-0.270124,-1.131578,-0.472311,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,0.0,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-0.059704,-0.075183,2018-11-15 03:34:21+00:00,-0.349252,-0.044962,0.175567,-0.008155,-0.404020,-0.008737,-0.240722,-0.027588,0.003014,3,15,11,2018,0.099615,0.099615,4.115579,4.170266,1.605500,0.0,2.733265,4.170420,-0.565355,2.705442,0


In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import mlflow
import mlflow.sklearn
from train import create_preprocessing_pipeline, train_model, evaluate_model, save_model

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.pipeline import Pipeline

Train size: (76529, 34)
Test size: (19133, 34)


In [7]:
# Logistic Regression pipeline
lr_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("classifier", LogisticRegression(max_iter=1000, random_state=42))
])

# Random Forest pipeline
rf_pipeline = Pipeline([
    ("classifier", RandomForestClassifier(random_state=42))
])


In [8]:
#  Grid Search for Random Forest
rf_params = {
    "classifier__n_estimators": [100, 200],
    "classifier__max_depth": [5, 10, None],
    "classifier__min_samples_split": [2, 5],
}

rf_grid = GridSearchCV(rf_pipeline, param_grid=rf_params, cv=3, scoring="roc_auc", n_jobs=-1)


In [9]:
mlflow.set_tracking_uri("file:./mlruns")
mlflow.set_experiment("credit-risk-task5")


c:\Users\User\anaconda3\Lib\site-packages\mlflow\tracking\_tracking_service\utils.py:178: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri, store_uri)
2025/12/30 01:14:32 INFO mlflow.tracking.fluent: Experiment with name 'credit-risk-task5' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///c:/Users/User/Desktop/KAIM/Week_4/credit-risk-model/notebooks/mlruns/235548982724386299', creation_time=1767046472792, experiment_id='235548982724386299', last_update_time=1767046472792, lifecycle_stage='active', name='credit-risk-task5', tags={}>

In [13]:
# Make project root visible so we can import src modules
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

project_root


'c:\\Users\\User\\Desktop\\KAIM\\Week_4\\credit-risk-model'

In [14]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import mlflow
import mlflow.sklearn
from src.train import create_preprocessing_pipeline, train_model, evaluate_model, save_model

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.pipeline import Pipeline

In [15]:
data_path = r"C:\Users\User\Desktop\KAIM\Week_4\credit-risk-model\Data\processed_data_with_target.csv"
df = pd.read_csv(data_path)

# Assume the target column is 'is_high_risk'
target_col = "is_high_risk"
X = df.drop(columns=[target_col])
y = df[target_col]


In [16]:
# ============================================================
# Train-test split
# ============================================================
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Train size: {len(X_train)}, Test size: {len(X_test)}")

# ============================================================
# Preprocessing pipeline
# ============================================================
preprocessor = create_preprocessing_pipeline(X_train)


2025-12-30 02:05:43,211 - INFO - Numeric columns: ['CountryCode', 'Amount', 'Value', 'PricingStrategy', 'FraudResult', 'total_amount', 'avg_amount', 'transaction_count', 'std_amount', 'total_value', 'avg_value', 'std_value', 'transaction_hour', 'transaction_day', 'transaction_month', 'transaction_year', 'TransactionId_woe', 'BatchId_woe', 'AccountId_woe', 'SubscriptionId_woe', 'CustomerId_woe', 'CurrencyCode_woe', 'ProviderId_woe', 'ProductId_woe', 'ProductCategory_woe', 'ChannelId_woe']
2025-12-30 02:05:43,222 - INFO - Categorical columns: ['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId', 'CurrencyCode', 'ProviderId', 'ProductId', 'ProductCategory', 'ChannelId', 'TransactionStartTime']


Train size: 76529, Test size: 19133


In [29]:
# ============================================================
# Define models
# ============================================================
MODEL_DICT = {
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42),
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "RandomForest": RandomForestClassifier(random_state=42),
    "GradientBoosting": GradientBoostingClassifier(random_state=42),
}

# ============================================================
# MLflow setup
# ============================================================
mlflow.set_tracking_uri("file:./notebooks_old/mlruns/1/models/")
mlflow.set_experiment("credit-risk-models")

<Experiment: artifact_location='file:///c:/Users/User/Desktop/KAIM/Week_4/credit-risk-model/notebooks/notebooks_old/mlruns/1/models/640032936158412455', creation_time=1767050605804, experiment_id='640032936158412455', last_update_time=1767050605804, lifecycle_stage='active', name='credit-risk-models', tags={}>

In [30]:
# ============================================================
# Train and track models
# ============================================================
results = {}

for model_name, model_instance in MODEL_DICT.items():
    with mlflow.start_run(run_name=model_name):
        # Create pipeline
        pipeline = Pipeline([
            ("preprocessor", preprocessor),
            ("classifier", model_instance)
        ])
        
        # Train
        pipeline.fit(X_train, y_train)
        
        # Predict
        y_pred = pipeline.predict(X_test)
        y_proba = pipeline.predict_proba(X_test)[:, 1] if hasattr(pipeline.named_steps['classifier'], "predict_proba") else y_pred
        
        # Metrics
        metrics = {
            "accuracy": accuracy_score(y_test, y_pred),
            "precision": precision_score(y_test, y_pred),
            "recall": recall_score(y_test, y_pred),
            "f1": f1_score(y_test, y_pred),
            "roc_auc": roc_auc_score(y_test, y_proba)
        }
        
        # Log metrics and model
        mlflow.log_params({"model_type": model_name})
        mlflow.log_metrics(metrics)
        mlflow.sklearn.log_model(pipeline, artifact_path="model")
        
        # Save locally if desired
        save_model(pipeline, filepath=f"notebooks_old/mlruns/1/models/{model_name}_pipeline.pkl")
        
        print(f"{model_name} metrics: {metrics}")
        results[model_name] = metrics


2025/12/30 02:25:00 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025-12-30 02:25:07,174 - INFO - Model saved to notebooks_old/mlruns/1/models/LogisticRegression_pipeline.pkl


LogisticRegression metrics: {'accuracy': 0.9924214707573302, 'precision': 0.9770642201834863, 'recall': 0.9573033707865168, 'f1': 0.9670828603859251, 'roc_auc': np.float64(0.998955510721605)}


2025/12/30 02:25:55 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025-12-30 02:26:01,670 - INFO - Model saved to notebooks_old/mlruns/1/models/DecisionTree_pipeline.pkl


DecisionTree metrics: {'accuracy': 0.9977525740866565, 'precision': 0.9901168014375562, 'recall': 0.990561797752809, 'f1': 0.99033924960683, 'roc_auc': np.float64(0.994630319269118)}


2025/12/30 02:30:47 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025-12-30 02:30:54,356 - INFO - Model saved to notebooks_old/mlruns/1/models/RandomForest_pipeline.pkl


RandomForest metrics: {'accuracy': 0.9911670935033712, 'precision': 0.9776951672862454, 'recall': 0.9456179775280898, 'f1': 0.9613890792780443, 'roc_auc': np.float64(0.9994378301076812)}


KeyboardInterrupt: 

In [31]:
# ============================================================
# Compare results
# ============================================================
results_df = pd.DataFrame(results).T
print("\nComparison of all models:")
print(results_df)


Comparison of all models:
                    accuracy  precision    recall        f1   roc_auc
LogisticRegression  0.992421   0.977064  0.957303  0.967083  0.998956
DecisionTree        0.997753   0.990117  0.990562  0.990339  0.994630
RandomForest        0.991167   0.977695  0.945618  0.961389  0.999438
